In [1]:
import numpy as np
np.random.seed(seed=28213)
import pandas as pd
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.svm import SVC
from scipy.stats import linregress
from allel import rogers_huff_r_between
from scipy.spatial.distance import squareform
from sklearn.preprocessing import normalize
from scipy.stats import norm
from scipy.stats import rankdata
import matplotlib as mpl
from matplotlib import rc,rcParams
from pylab import *
mpl.use("pgf")
# activate latex text rendering
rc('text', usetex=True)
rc('axes', linewidth=2)
rc('font', weight='bold')
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'text.latex.preamble':r'\usepackage{sfmath} \boldmath'
})
import matplotlib.pyplot as plt
import matplotlib.pyplot as figure
import seaborn as sns
from seaborn import lineplot
sns.set_theme(style="whitegrid")

In [2]:
bloom_loci = pd.read_csv('/Users/vicious/Documents/ShiLab/PhenotypePrediction_GA/bloom_detected_QTL.csv')

In [5]:
genotype_file = '../data/genotype_full.txt'
phenotype_file = '../data/phenotype.csv'
def feature_ranking(score):
    """
    Rank features in descending order according to their score, the larger the score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices
genotypes = pd.read_csv(genotype_file, sep='\t', index_col=0)
genotypes[genotypes == -1] = 0
multi_pheno = pd.read_csv(phenotype_file, sep=',', index_col=0)

## Direct Intersections

In [9]:
for phenoIndex in range(10):
    print()
    phenoName = multi_pheno.columns[phenoIndex]
    print(phenoIndex,">",phenoName, ":")
    genes0 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.2_Intersection.csv')
    genes1 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.3_Intersection.csv')
    genes2 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.4_Intersection.csv')
    genes3 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.5_Intersection.csv')
    sets = {0.2:genes0, 0.3:genes1, 0.4:genes2, 0.5:genes3}
    LD_generator = np.linspace(0.0, 1.0, num=100)
    for (key, ggg) in sets.items():
        g_expanded = set(list(np.where(ggg.to_numpy() == 1)[1]))
        generator = set(bloom_loci[~bloom_loci.iloc[:, phenoIndex].isna()].iloc[:, phenoIndex].to_numpy().astype(int))
        print(key, '>', g_expanded.intersection(generator))
        


0 > 1_CobaltChloride_1 :
0.2 > {15104, 9825, 27047, 3210, 9358, 7022, 20567, 19032}
0.3 > {15104, 9825, 13606, 27047, 26569, 3210, 9358, 7022, 20567, 19032}
0.4 > {15104, 9825, 27047, 26569, 3210, 9358, 7022, 20567, 19032}
0.5 > {15104, 9825, 27047, 26569, 3210, 9358, 7022, 14165, 20567, 19032}

1 > 1_CopperSulfate_1 :
0.2 > {11460, 17512, 20655, 19024, 4976, 25746, 12254}
0.3 > {11460, 17512, 20655, 19024, 4976, 25746, 12254}
0.4 > {11460, 17512, 20655, 19024, 4976, 25746, 12254}
0.5 > {11460, 17512, 20655, 19024, 4976, 25746, 12254}

2 > 1_Diamide_1 :
0.2 > {11144, 17729, 23847}
0.3 > {11144, 17729, 23847, 18575}
0.4 > {17729, 23847, 11144, 12491, 18575}
0.5 > {17729, 23847, 11144, 12491, 18575}

3 > 1_E6-Berbamine_1 :
0.2 > {4994, 27818, 23469, 4662, 12694, 28089, 574}
0.3 > {4994, 23469, 4662, 12694, 28089, 574}
0.4 > {4994, 23469, 4662, 12694, 28089, 574}
0.5 > {4994, 20872, 23469, 4662, 12694, 28089, 574}

4 > 1_Ethanol_1 :
0.2 > {16187, 4834, 5033, 20655, 25397, 19064, 1691}
0.

## LD Concordance

In [5]:
# plt.figure(num=None, figsize=(20, 12), dpi=80, facecolor='w', edgecolor='k')
fig, axs = plt.subplots(4, 3, figsize=(22,12), dpi=400, constrained_layout=True)
# fig.tight_layout()
# plt.subplots_adjust(hspace = .5)
for phenoIndex in range(10):
    print(phenoIndex)
    phenoName = multi_pheno.columns[phenoIndex]
    print(phenoName)
    phenotypes = multi_pheno.iloc[:, phenoIndex]
    y = phenotypes
    x = genotypes.to_numpy()

    results =[]
    
    genes0 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.2_Intersection.csv')
    genes1 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.3_Intersection.csv')
    genes2 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.4_Intersection.csv')
    genes3 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.5_Intersection.csv')


    sets = {0.2:genes0, 0.3:genes1, 0.4:genes2, 0.5:genes3}
    LD_generator = np.linspace(0.0, 1.0, num=100)
    res = []
    res_dummy = []
    for (key, ggg) in sets.items():
        g_expanded = list(np.where(ggg.to_numpy() == 1)[1])
        generator = set(bloom_loci[~bloom_loci.iloc[:, phenoIndex].isna()].iloc[:, phenoIndex].to_numpy().astype(int))
        for r2_threshold in sorted(LD_generator, reverse=True):
            
            r = rogers_huff_r_between(x[:, list(sorted(generator))].transpose(), x[:, g_expanded].transpose())
            r2 = r ** 2
            
            for i, ind in enumerate(list(sorted(generator))):
                if (r2[i, :] > r2_threshold).any():
                    generator.remove(ind)
                    res.append(r2_threshold)
                    res_dummy.append(key)
            
    results.append(res)
    results.append(res_dummy)

    names = ["y", "x"]
    results = np.array(results)
    results = pd.DataFrame(data=results.T, columns=names, dtype=np.float32, copy=False)


    ax = sns.violinplot(x='x', y='y', data=results,
                        palette="flare", ax=axs[phenoIndex//3, phenoIndex%3] if phenoIndex!=9 else axs[3,1],
                        bw=.1, cut=0)
    ax.set_ylim([0, 1.1])
    ax.set_title(phenoName[2:-2], x=0.87, fontweight='bold', fontsize=17)

fig.delaxes(axs[3,0])
fig.delaxes(axs[3,2])
for ax in axs.flat:
    ax.set_xlabel(r'\textbf{LD threshold}', fontsize=15)
    ax.set_ylabel(r'\textbf{LD concordance}', fontsize=15)
    ax.xaxis.set_tick_params(labelsize=15)
    ax.yaxis.set_tick_params(labelsize=15)

for i in range (1, 3):
    for j in range(2):
        axs[j, i].label_outer()
        axs[j, i].xaxis.set_ticklabels([])
        axs[j, i].yaxis.set_ticklabels([])
        axs[j, i].set_xlabel('')
        axs[j, i].set_ylabel('')
        
axs[2, 1].label_outer()
axs[2, 1].xaxis.set_ticklabels([])
axs[2, 1].yaxis.set_ticklabels([])
axs[2, 1].set_xlabel('')
axs[2, 1].set_ylabel('')

axs[2, 2].yaxis.set_ticklabels([])
axs[2, 2].set_ylabel('')

for i in range(2):
    axs[i, 0].xaxis.set_ticklabels([])
    axs[i, 0].set_xlabel('')

plt.subplots_adjust(wspace=0.1)


# Hide x labels and tick labels for top plots and y ticks for right plots.
# for ax in axs.flat:
#     ax.label_outer()
    
plt.show()


0
1_CobaltChloride_1
1
1_CopperSulfate_1
2
1_Diamide_1
3
1_E6-Berbamine_1
4
1_Ethanol_1
5
1_Formamide_1
6
1_Hydroxyurea_1
7
1_IndolaceticAcid_1
8
1_Lactate_1
9
1_Lactose_1


<ipython-input-5-fb2ec6b80ecc>:82: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  plt.subplots_adjust(wspace=0.1)
<ipython-input-5-fb2ec6b80ecc>:89: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [7]:
# plt.figure(num=None, figsize=(20, 12), dpi=80, facecolor='w', edgecolor='k')
fig, axs = plt.subplots(5, 2, figsize=(15,22), dpi=400, constrained_layout=True)
# fig.tight_layout()
# plt.subplots_adjust(hspace = .5)
for phenoIndex in range(10):
    print(phenoIndex)
    phenoName = multi_pheno.columns[phenoIndex]
    print(phenoName)
    phenotypes = multi_pheno.iloc[:, phenoIndex]
    y = phenotypes
    x = genotypes.to_numpy()

    results =[]
    
    genes0 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.2_Intersection.csv')
    genes1 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.3_Intersection.csv')
    genes2 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.4_Intersection.csv')
    genes3 = pd.read_csv(f'experimentResults/PearsonCC_{phenoName}_LD_0.5_Intersection.csv')


    sets = {0.2:genes0, 0.3:genes1, 0.4:genes2, 0.5:genes3}
    LD_generator = np.linspace(0.0, 1.0, num=100)
    res = []
    res_dummy = []
    for (key, ggg) in sets.items():
        g_expanded = list(np.where(ggg.to_numpy() == 1)[1])
        generator = set(bloom_loci[~bloom_loci.iloc[:, phenoIndex].isna()].iloc[:, phenoIndex].to_numpy().astype(int))
        for r2_threshold in sorted(LD_generator, reverse=True):
            
            r = rogers_huff_r_between(x[:, list(sorted(generator))].transpose(), x[:, g_expanded].transpose())
            r2 = r ** 2
            
            for i, ind in enumerate(list(sorted(generator))):
                if (r2[i, :] > r2_threshold).any():
                    generator.remove(ind)
                    res.append(r2_threshold)
                    res_dummy.append(key)
            
    results.append(res)
    results.append(res_dummy)

    names = ["y", "x"]
    results = np.array(results)
    results = pd.DataFrame(data=results.T, columns=names, dtype=np.float32, copy=False)


    ax = sns.violinplot(x='x', y='y', data=results,
                        palette="flare", ax=axs[phenoIndex//2, phenoIndex%2],
                        bw=.1, cut=0)
    ax.set_ylim([0, 1.1])
    ax.set_title(phenoName[2:-2], x=0.87, fontweight='bold', fontsize=17)

# fig.delaxes(axs[3,0])
# fig.delaxes(axs[3,2])
for ax in axs.flat:
    ax.set_xlabel(r'\textbf{LD threshold}', fontsize=15)
    ax.set_ylabel(r'\textbf{LD concordance}', fontsize=15)
    ax.xaxis.set_tick_params(labelsize=15)
    ax.yaxis.set_tick_params(labelsize=15)

# for i in range (1, 3):
#     for j in range(2):
#         axs[j, i].label_outer()
#         axs[j, i].xaxis.set_ticklabels([])
#         axs[j, i].yaxis.set_ticklabels([])
#         axs[j, i].set_xlabel('')
#         axs[j, i].set_ylabel('')
        
# axs[2, 1].label_outer()
# axs[2, 1].xaxis.set_ticklabels([])
# axs[2, 1].yaxis.set_ticklabels([])
# axs[2, 1].set_xlabel('')
# axs[2, 1].set_ylabel('')

# axs[2, 2].yaxis.set_ticklabels([])
# axs[2, 2].set_ylabel('')

# for i in range(2):
#     axs[i, 0].xaxis.set_ticklabels([])
#     axs[i, 0].set_xlabel('')

# plt.subplots_adjust(wspace=0.1)


# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()
    
plt.show()



0
1_CobaltChloride_1
1
1_CopperSulfate_1
2
1_Diamide_1
3
1_E6-Berbamine_1
4
1_Ethanol_1
5
1_Formamide_1
6
1_Hydroxyurea_1
7
1_IndolaceticAcid_1
8
1_Lactate_1
9
1_Lactose_1


<ipython-input-7-c08922bc2d1f>:89: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [10]:
plt.savefig("bloom_concordance.png", bbox_inches='tight', format="png", dpi=400)

In [35]:
plt.savefig("bloom_concordance.pgf", bbox_inches='tight', format="pgf", dpi=400)

In [11]:
plt.savefig("bloom_concordance.svg", bbox_inches='tight', format="svg", dpi=400)

In [8]:
plt.savefig("Figure 1.pdf", bbox_inches='tight', format="pdf", dpi=600)